# Pythonでデータ解析
　ここでは、遺伝子発現量ファイルをPythonプログラムで扱う練習をします。ファイルの読み込み、表示、条件にあうデータ行の抽出をおこないます。また、データのグラフ化、統計処理もおこなってみます。  
　サンプルデータは、赤シソと青シソの遺伝子発現量のデータです。  

---

## 0. Jupyter Notebookの基本
- 現在のセルのプログラムを実行する: __Ctrl + Enter__    

- 現在のセルのプログラムを実行し、次のセルに移る: __Shift + Enter__  
- 直前の動作をやり直す: __Ctrl + Z__ (Macの場合 __command + Z__)    
誤ってプログラムを消してしまった場合によく使います。  
- その他のショートカットを知りたい場合:  __Help > Keyboard Shortcuts__   
- ファイルの名前を変更する: __File > Rename__

In [2]:
# 必要なライブラリをインポートする
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

## 1. テキストファイル読み込み・表示
[読み込むファイル]  
* ./05_cuffdiff_out/gene_exp.diff

[使用する関数] 
* ファイルを開く _open()_  
* ファイルを閉じる _close()_ 
* 一行読み込む _readline()_  
* 全ての行を読み込む _readlines()_  
* 表示する _print()_  
* 繰り返し処理をする _for ~ in ...:_  
* 条件分岐させる　_if ~: ... else: ..._ 
* 改行コードを除去する _rstrip()_   
* 行をタブコードで分割する: _split('\t')_

[例]  

```python
f = open('./05_cuffdiff_out/gene_exp.diff', 'r')
lines = f.readlines()
f.close()

for line in lines:
    line = line.rstrip()
    items = line.split('\t')
    if 'yes' == items[-1]:
        print('This is "yes" line')
    else:
        print('This is "no" line')
```

---
# 2. データフレームを使う
[データブレーム]  
- Excelのような行列表  

[使用する関数]　　
- データフレームとしてファイルを読み込む __pd.read_csv(FILE_PATH, sep=..., header=...)__  
- 値の数をカウントする __value_counts()__  
- 条件にマッチするデータのみ表示する __isin()__
- 指定した列のデータを表示する __df.loc__

[例]

```python
# データフレームとしてファイルを読み込む
diff = './05_cuffdiff_out/gene_exp.diff'
df = pd.read_csv(diff, sep='\t', header=0)
df
```

---

```python
# 指定した行のデータを表示する
df[1:3]

# 指定した列のデータを表示する
df.loc[:,['locus', 'sample_1', 'sample_2', 'value_1','value_2', 'q_value', 'significant']]
```

---

```python
# 値の数をカウントする
df['significant'].value_counts()
```

---

```python
# 条件にマッチするデータのみ表示する
sig_yes = df[df['significant'].isin(['yes'])]
sig_yes

#sig_no = df[df['significant'].isin(['no'])]
```

## 3. グラフ表示
__plt.figure()__, __add_subplot__ など

In [38]:
# 各座標の準備
x = df['value_1']
y = df['value_2']

# 図の準備
#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)

# 座標データをプロットする
#ax.scatter(x,y, c='silver')

# 図の装飾（タイトルや軸ラベルなど）
#ax.set_title('Gene Expression Plot')
#ax.set_xlabel('Gene Expression (FPKM) of red perilla')
#ax.set_ylabel('Gene Expression (FPKM) of green perilla')
#plt.style.use('ggplot')

# 図の描画
#fig.show()

[例1] 散布図

```
# 各座標の準備
# "yes"データの(x座標, y座標)=(value_1, value_2)を取得する。
x1 = sig_yes['value_1']
y1 = sig_yes['value_2']
# "no"データの(x座標, y座標)=(value_1, value_2)を取得する。
x2 = sig_no['value_1']
y2 = sig_no['value_2']

# 対数値に変換 (log)
x1 = np.log10(x1 + 10**(-10))
y1 = np.log10(y1 + 10**(-10))
x2 = np.log10(x2 + 10**(-10))
y2 = np.log10(y2 + 10**(-10))

# 図の準備
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# 座標データをプロットする
ax.scatter(x2,y2, c='silver')
ax.scatter(x1,y1, c='red')

# 図の装飾（タイトルや軸ラベルなど）
ax.set_title('Gene Expression Plot')
ax.set_xlabel('Gene Expression (FPKM) of red perilla')
ax.set_ylabel('Gene Expression (FPKM) of green perilla')
plt.style.use('ggplot')

# 図の描画
fig.show()
```
---
![fpkm_scatterplot](./etc/fpkm_scatterplot.png "FPKM scatter plot")

[例2] 箱ひげ図

```python
# データ準備
red = list(np.log10(df['value_1'] + 10**(-10)))
green = list(np.log10(df['value_2'] + 10**(-10)))

# 図の準備
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# 箱ひげ図にデータをプロットする
cmp = (red, green)
bp = ax.boxplot(cmp)

# 図の装飾
plt.title('Box plot of Gene Expression')
plt.ylabel('log10 of FPKM')
ax.set_xticklabels(['red perilla', 'green perilla'])
plt.style.use('ggplot')

# 図の描画
plt.show()
```
---
![fpkm_boxplot](./etc/fpkm_boxplot.png "FPKM box plot")

## 4. 統計処理
対応のある２群のt検定 __stats.ttest_rel()__

In [14]:
df.loc[:,['locus', 'sample_1', 'sample_2', 'value_1','value_2', 'q_value', 'significant']]

,locus,sample_1,sample_2,value_1,value_2,q_value,significant
0,c10478_g1_i1:4-878,red_perilla,green_perilla,31265.30,39514.70,0.096678,no
1,c1235_g1_i1:0-358,red_perilla,green_perilla,18899.00,25190.00,0.450315,no
2,c14532_g1_i1:672-1250,red_perilla,green_perilla,1036.71,6905.89,0.098630,no
3,c14964_g1_i1:0-1591,red_perilla,green_perilla,96498.60,106401.00,0.096678,no
4,c18250_g2_i1:33-1490,red_perilla,green_perilla,139165.00,27992.50,0.000175,yes
5,c18250_g2_i2:33-1519,red_perilla,green_perilla,135968.00,24295.30,0.000175,yes
6,c20303_g1_i1:0-1582,red_perilla,green_perilla,255968.00,55893.00,0.000175,yes
7,c2030_g1_i1:0-433,red_perilla,green_perilla,21027.10,10418.10,0.096678,no
8,c21075_g2_i1:0-1482,red_perilla,green_perilla,485920.00,107958.00,0.000175,yes
9,c22175_g1_i1:0-492,red_perilla,green_perilla,16423.80,29570.80,0.076160,no


In [15]:
red = list(df['value_1'])
green = list(df['value_2'])
res = stats.ttest_rel(red, green)
res

Ttest_relResult(statistic=1.8679404345702795, pvalue=0.084472971958650486)